In [ ]:
#載入所需函示庫
from matplotlib import pyplot as plt
%matplotlib inline
# 設定圖形大小; DPI越大圖越大
plt.rcParams["figure.dpi"] = 100
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as smm
import statsmodels.stats.outliers_influence as sso
import math
import statistics
from statsmodels.tsa.api import Holt
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt

In [ ]:
def SI_LR(y_v, n):
    y_data = y_v
    X_data_ar = np.arange(1, len(y_v)+1)
    X_data_T = X_data_ar.T
    X_data = pd.DataFrame(X_data_T,columns = ['Time'])
    X_data = sm.add_constant(X_data)
    olsmod = sm.OLS(y_data, X_data)
    result_reg = olsmod.fit()
    st, data, ss2 = sso.summary_table(result_reg, alpha=0.05)
    y_v_LR_a = data[:, 2]
    SI_LR_a = y_v / y_v_LR_a
    SI_id_s = np.arange(1,len(y_v)+1)
    SI_id = SI_id_s - np.floor(SI_id_s / n) * n
    SI_id[np.where((SI_id[:] == 0))] = n
    SI_LR_a_df = pd.DataFrame({'SILR': SI_LR_a, 'SIid': SI_id})
    SI_LR_u = np.zeros(n)
    for j in range(1,n+1):
        SI_LR_u[j-1] = SI_LR_a_df['SILR'][SI_LR_a_df['SIid'] == j].dropna().mean()
    SI_LR = SI_LR_u / sum(SI_LR_u) * n
    print('Seasonal Index:', SI_LR)
    print('\n')
    y_v_SI_LR = np.zeros(len(y_v))
    DSI_y_v = np.zeros(len(y_v))
    for k in range(0,len(y_v)):
        Idd = int(SI_LR_a_df['SIid'][k] - 1)
        y_v_SI_LR[k] = SI_LR[Idd]
        DSI_y_v[k] = y_v[k] / SI_LR[Idd]
    print('Deseasonalized Data:', DSI_y_v)
    SI_LR_result_a = np.array([SI_LR_a_df['SIid'], y_v_SI_LR, y_v, DSI_y_v])
    SI_LR_result_df = pd.DataFrame(SI_LR_result_a.T, columns = ['SID', 'SeaIdx', 'orig', 'Des_D'])
    return {"LR_ind":SI_LR, "SI_LR_res":SI_LR_result_df}

In [ ]:
def runsTest(l, l_median): 
    runs, n1, n2 = 1, 0, 0
    if(l[0]) >= l_median:
        n1 += 1   
    else:
        n2 += 1   
    for i in range(1, len(l)): 
        if (l[i] >= l_median and l[i-1] < l_median) or (l[i] < l_median and l[i-1] >= l_median):
            runs += 1  
        if(l[i]) >= l_median: 
            n1 += 1   
        else: 
            n2 += 1   
    runs_exp = ((2*n1*n2)/(n1+n2)) + 1
    stan_dev = math.sqrt((2*n1*n2*(2*n1*n2-n1-n2))/(((n1+n2)**2)*(n1+n2-1))) 
    z = (runs-runs_exp)/stan_dev
    pval_z = stats.norm.sf(abs(z)) * 2
    print('runs = ', runs)
    print('n1 = ', n1)
    print('n2 = ', n2)
    print('runs_exp = ', runs_exp)
    print('stan_dev = ', stan_dev)
    print('z = ', z)
    print('pval_z = ', pval_z)
    return pval_z

In [ ]:
def Durbin_Watson_test(x):
    x_square_sum = np.vdot(x, x)
    print("x_square_sum = ", x_square_sum)
    size = x.size
    print("size = ", size)
    x_d = np.zeros((size))
    #print("x_d = ", x_d)
    l_size = size - 1
    for i in range(l_size):
        x_d[i + 1] = x[i + 1] - x[i]
    print("x_d = ", x_d)
    d = np.vdot(x_d, x_d) / x_square_sum
    print("d = ", d)
    return(d)

In [ ]:
def SI_MA(y_v, n):
    y_v_MA_a = np.zeros(len(y_v))
    y_v_MA_a[:] = np.nan
    mvlen = n
    if n % 2 == 0:
        halfwin = int(mvlen / 2)
        y_v_MA_ta = np.zeros(len(y_v))
        for i in range(halfwin, len(y_v) - halfwin + 1):
            y_v_MA_ta[i] = np.mean(y_v[(i-halfwin): (i+halfwin)])
        for ii in range(halfwin, len(y_v) - halfwin):
            y_v_MA_a[ii] = np.mean(y_v_MA_ta[(ii): (ii+2)])
    else:
        halfwin = int((mvlen - 1) / 2)
        for i in range(halfwin, len(y_v) - halfwin):
            y_v_MA_a[i] = np.mean(y_v[(i-halfwin): (i+halfwin+1)])
    SI_MA_a = np.zeros(len(y_v))
    SI_MA_a[:] = np.nan
    SI_MA_a = y_v / y_v_MA_a    
    SI_id_s = np.arange(1,len(y_v)+1)
    SI_id = SI_id_s - np.floor(SI_id_s / n) * n
    SI_id[np.where((SI_id[:] == 0))] = n
    SI_MA_a_df = pd.DataFrame({'SIMA': SI_MA_a, 'SIid': SI_id})
    SI_MA_u = np.zeros(n)
    for j in range(1,n+1):
        SI_MA_u[j-1] = SI_MA_a_df['SIMA'][SI_MA_a_df['SIid'] == j].dropna().mean()  
    SI_MA = SI_MA_u / sum(SI_MA_u) * n
    print('Seasonal Index:', SI_MA ) 
    y_v_SI_MA = np.zeros(len(y_v))
    DSI_y_v = np.zeros(len(y_v))
    for k in range(0,len(y_v)):
        Idd = int(SI_MA_a_df['SIid'][k] - 1)
        y_v_SI_MA[k] = SI_MA[Idd]
        DSI_y_v[k] = y_v[k] / SI_MA[Idd]
    print('Deseasonalized Data:', DSI_y_v)
    SI_MA_result_a = np.array([SI_MA_a_df['SIid'], y_v_SI_MA, y_v, DSI_y_v])
    SI_MA_result_df = pd.DataFrame(SI_MA_result_a.T, columns = ['SID', 'SeaIdx', 'orig', 'Des_D'])
    return {"ma_ind":SI_MA, "SI_MA_res":SI_MA_result_df}

In [ ]:
def ErrMetrix(y_v, f_v):
    y_v_n = pd.DataFrame({'orig': y_v})
    f_v_n = pd.DataFrame({'forecast': f_v})
    err1 = (y_v_n['orig'] - f_v_n['forecast']).dropna().values
    err1p = ((y_v_n['orig'] - f_v_n['forecast'])/ y_v_n['orig']).dropna().values
    MAD = round(np.absolute(err1).mean(),6)
    MSE = round(np.mean(err1 * err1),6)
    RMSE = round(np.sqrt(np.mean(err1 * err1)),6)
    MAPE = round(np.absolute(err1p).mean() * 100, 6)
    print("MAD = ", MAD)
    print("MSE = ", MSE)
    print("RMSE = ", RMSE)
    print("MAPE = ", MAPE, "%")
    return MAD, MSE, RMSE, MAPE

In [ ]:
#讀取資料集
df1 = pd.read_csv('excel/110.csv')
print('Head of dataset:')
display(df1.head())
print("Tail of dataset:")
display(df1.tail())

In [ ]:
for i in range(365):
    df1['水庫進流量/立方公尺'][i] = int(df1['水庫進流量/立方公尺'][i].replace(",",""))

In [ ]:
mean_1 = np.zeros(12)
print(mean_1)
for i in range(1, 13):
    #print(i)
    if (i == 1) or (i == 2): 
        #print(i)
        filt = (df1['日期'].str.contains(str(i)+'月', na=False)) & ~ (df1['日期'].str.contains('1'+str(i)+'月', na=False))
    else:
        filt = (df1['日期'].str.contains(str(i)+'月', na=False))
    df = df1.loc[filt, ['日期', '水庫進流量/立方公尺']]
    mean_1[i-1] = np.mean(df['水庫進流量/立方公尺'])

print(mean_1)
#print(df['水庫進流量/立方公尺'])
#print(df1.loc[filt, ['日期', '水庫進流量/立方公尺']])

In [ ]:
#讀取資料集
df2 = pd.read_csv('excel/109.csv')
print('Head of dataset:')
display(df2.head())
print("Tail of dataset:")
display(df2.tail())

In [ ]:
for i in range(366):
    df2['水庫進流量/立方公尺'][i] = int(df2['水庫進流量/立方公尺'][i].replace(",",""))

In [ ]:
mean_2 = np.zeros(12)
print(mean_2)
for i in range(1, 13):
    #print(i)
    if (i == 1) or (i == 2): 
        #print(i)
        filt = (df2['日期'].str.contains(str(i)+'月', na=False)) & ~ (df2['日期'].str.contains('1'+str(i)+'月', na=False))
    else:
        filt = (df2['日期'].str.contains(str(i)+'月', na=False))
    df = df2.loc[filt, ['日期', '水庫進流量/立方公尺']]
    mean_2[i-1] = np.mean(df['水庫進流量/立方公尺'])

print(mean_2)
#print(df['水庫進流量/立方公尺'])
#print(df1.loc[filt, ['日期', '水庫進流量/立方公尺']])

In [ ]:
#讀取資料集
df3 = pd.read_csv('excel/108.csv')
print('Head of dataset:')
display(df3.head())
print("Tail of dataset:")
display(df3.tail())

In [ ]:
for i in range(365):
    df3['水庫進流量/立方公尺'][i] = int(df3['水庫進流量/立方公尺'][i].replace(",",""))

In [ ]:
mean_3 = np.zeros(12)
print(mean_3)
for i in range(1, 13):
    #print(i)
    if (i == 1) or (i == 2): 
        #print(i)
        filt = (df3['日期'].str.contains(str(i)+'月', na=False)) & ~ (df3['日期'].str.contains('1'+str(i)+'月', na=False))
    else:
        filt = (df3['日期'].str.contains(str(i)+'月', na=False))
    df = df3.loc[filt, ['日期', '水庫進流量/立方公尺']]
    mean_3[i-1] = np.mean(df['水庫進流量/立方公尺'])

print(mean_3)
#print(df['水庫進流量/立方公尺'])
#print(df1.loc[filt, ['日期', '水庫進流量/立方公尺']])

In [ ]:
mean = np.concatenate((mean_1, mean_2, mean_3), axis=None)
#print(mean)

df_1 = pd.DataFrame(mean, columns = ['mean'])
print('Head of dataset:')
display(df_1.head())
print("Tail of dataset:")
display(df_1.tail())

In [ ]:
t = range(1,df_1.shape[0]+1)
df_1.insert(0, 'Period', t)
#display(df_1)

In [ ]:
t = range(0,df_1.shape[0])
df_1.insert(2, 't', t)
#display(df_1)

In [ ]:
plt.scatter(df_1['Period'], df_1['mean'], color='blue')
plt.plot(df_1['Period'], df_1['mean'], color='blue')
sns.regplot(x = df_1['Period'], y = df_1['mean'], data = df_1, color = 'r', ci = None, scatter = False)
plt.ylim(0, max(df_1['mean'])+5)
plt.title('Line Plot for mean against period')
plt.xlabel('period')
plt.ylabel('mean')
plt.show()

In [ ]:
df_1_ts3_MA_S_result = pd.DataFrame(columns=["ma_ind", "SI_MA_res"])
df_1_ts3_MA_S_result = SI_MA(df_1['mean'], 12)
df_1_ts3_MA_Idx = df_1_ts3_MA_S_result["ma_ind"]
print(df_1_ts3_MA_Idx)
df_1_ts3_MA_Result = df_1_ts3_MA_S_result["SI_MA_res"]
display(df_1_ts3_MA_Result)
plt.plot(df_1['t'], df_1_ts3_MA_Result['orig'])
plt.plot(df_1['t'], df_1_ts3_MA_Result['Des_D'])
plt.legend(labels=['org', 'Des_data'])
plt.xlabel('time')
plt.ylabel('Sales')
plt.title('Sales by Moving Average Seasonal Index')
plt.show()

In [ ]:
y_data_df_1_ts3 = df_1_ts3_MA_Result['Des_D']
X_data_ar_df_1_ts3 = np.array([df_1['t']])
X_data_ar_df_1_ts3_T = X_data_ar_df_1_ts3.T
X_data_df_1_ts3 = pd.DataFrame(X_data_ar_df_1_ts3_T, columns = ['t'])
X_data_df_1_ts3 = sm.add_constant(X_data_df_1_ts3)
olsmod_df_1_ts3_MA = sm.OLS(y_data_df_1_ts3, X_data_df_1_ts3)
result_reg_df_1_ts3_MA = olsmod_df_1_ts3_MA.fit()
print(result_reg_df_1_ts3_MA.summary())
df_1_ts3_MA_st, df_1_ts3_MA_data, df_1_ts3_MA_ss2 = sso.summary_table(result_reg_df_1_ts3_MA, alpha=0.05)
df_1_ts3_MA_y_pre = df_1_ts3_MA_data[:, 2]
des_pre_df_1_ts3_MA = df_1_ts3_MA_y_pre
print(des_pre_df_1_ts3_MA)

In [ ]:
plt.plot(df_1['t'], df_1_ts3_MA_Result['orig'])
plt.plot(df_1['t'], df_1_ts3_MA_Result['Des_D'])
plt.plot(df_1['t'], des_pre_df_1_ts3_MA)
plt.legend(labels=['org', 'Des_data', 'Line_pre'])
plt.xlabel('time')
plt.ylabel('Sales')
plt.title('Sales by Moving Average Seasonal Index')
plt.show()

In [ ]:
df_1_ts3_MA_st, df_1_ts3_MA_data, df_1_ts3_MA_ss2 = sso.summary_table(result_reg_df_1_ts3_MA, alpha=0.05)
df_1_ts3_MA_y_pre = df_1_ts3_MA_data[:, 2]
df_1_ts3_MA_SD = df_1_ts3_MA_data[:, 10]
print(df_1_ts3_MA_SD)
plt.plot(df_1_ts3_MA_y_pre, df_1_ts3_MA_SD, 'o', color = 'gray')
plt.axhline(y=2, color = 'red', lw = 0.8)
plt.axhline(y=0, color = 'blue')
plt.axhline(y=-2, color = 'red', lw = 0.8)
plt.title('Standardized Residual Plot')
plt.xlabel('Predicted y value')
plt.ylabel('Standardized Residual')
plt.show()

In [ ]:
print('Shapiro Test')
stat, p = stats.shapiro(df_1_ts3_MA_SD)
print('Statistics=%.3f, p=%.3f' % (stat, p))
df_1_ts3_MA_SD_median = statistics.median(df_1_ts3_MA_SD)
#print('median = ', SD_median)
Z_pval = runsTest(df_1_ts3_MA_SD, df_1_ts3_MA_SD_median) 
print('p_value for Z-statistic= ', Z_pval)
print('\n')
print('The Durbin Watson test')
d_w = Durbin_Watson_test(df_1_ts3_MA_SD)
print('d value =', d_w)

In [ ]:
df_1['Y_t1'] = df_1['mean'].shift(periods = 1)
df_1_y1 = df_1.dropna()
print(df_1_y1.shape)
y_var = df_1_y1['mean'].values
x_var_1 = df_1_y1['Y_t1'].values
sns.regplot(x = x_var_1, y = y_var, data = df_1_y1, color = 'r', ci = None)
plt.title('Line Plot for Percent_Change')
plt.xlabel('Y_t1')
plt.ylabel('Percent_Change')
plt.show()

In [ ]:
df_1_mod = AutoReg(df_1['mean'], 1, old_names=False)
df_1_res = df_1_mod.fit()
print(df_1_res.summary())

In [ ]:
df_1_ts3_MA_st, df_1_ts3_MA_data, df_1_ts3_MA_ss2 = sso.summary_table(result_reg_df_1_ts3_MA, alpha=0.05)
df_1_ts3_MA_y_pre = df_1_ts3_MA_data[:, 2]
df_1_ts3_MA_SD = df_1_ts3_MA_data[:, 10]
print(df_1_ts3_MA_SD)
plt.plot(df_1_ts3_MA_y_pre, df_1_ts3_MA_SD, 'o', color = 'gray')
plt.axhline(y=2, color = 'red', lw = 0.8)
plt.axhline(y=0, color = 'blue')
plt.axhline(y=-2, color = 'red', lw = 0.8)
plt.title('Standardized Residual Plot')
plt.xlabel('Predicted y value')
plt.ylabel('Standardized Residual')
plt.show()

In [ ]:
print('Shapiro Test')
stat, p = stats.shapiro(df_1_ts3_MA_SD)
print('Statistics=%.3f, p=%.3f' % (stat, p))
df_1_ts3_MA_SD_median = statistics.median(df_1_ts3_MA_SD)
#print('median = ', SD_median)
Z_pval = runsTest(df_1_ts3_MA_SD, df_1_ts3_MA_SD_median) 
print('p_value for Z-statistic= ', Z_pval)
print('\n')
print('The Durbin Watson test')
d_w = Durbin_Watson_test(df_1_ts3_MA_SD)
print('d value =', d_w)

In [ ]:
Sea_pre_df_1_ts3_MA = des_pre_df_1_ts3_MA * df_1_ts3_MA_Result['SeaIdx']
New_t = np.array([36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])
Total_t = np.append(df_1['t'], New_t)
New_t = sm.add_constant(New_t)
df_1_ts3_MA_des_pre_New_t = result_reg_df_1_ts3_MA.predict(New_t)
print('New Des Prediction', df_1_ts3_MA_des_pre_New_t)
df_1_ts3_MA_Sea_pre_New_t = df_1_ts3_MA_des_pre_New_t * df_1_ts3_MA_Idx
print('New Prediction', df_1_ts3_MA_Sea_pre_New_t)
New_Nan = np.zeros(New_t.shape[0])
New_Nan[:] = np.nan
df_1_ts3_MA_New_orig = np.append(df_1_ts3_MA_Result['orig'], New_Nan)
df_1_ts3_MA_New_Des_D = np.append(df_1_ts3_MA_Result['Des_D'], New_Nan)
df_1_ts3_MA_New_des_pre = np.append(des_pre_df_1_ts3_MA, df_1_ts3_MA_des_pre_New_t)
df_1_ts3_MA_New_Sea_pre = np.append(Sea_pre_df_1_ts3_MA, df_1_ts3_MA_Sea_pre_New_t)
df_1_ts3_MA_S_a = np.array([Total_t, df_1_ts3_MA_New_orig, df_1_ts3_MA_New_Des_D, df_1_ts3_MA_New_des_pre, df_1_ts3_MA_New_Sea_pre])
df_1_ts3_MA_S_df = pd.DataFrame(df_1_ts3_MA_S_a.T, columns=['time', 'org', 'Des_data', 'Line_pre', 'Seasonal Pre'])
display(df_1_ts3_MA_S_df)
plt.plot(Total_t, df_1_ts3_MA_New_Des_D)
plt.plot(Total_t, df_1_ts3_MA_New_des_pre)
plt.plot(Total_t, df_1_ts3_MA_New_Sea_pre)
#plt.ylim(0, MA_Result['orig'])+5)
plt.legend(labels=['org', 'Des_data', 'Line_pre', 'Seasonal Pre'])
plt.xlabel('time')
plt.ylabel('Sales')
plt.title('Sales by Moving Average Seasonal Index')
plt.show()

# regress model

In [ ]:
# Read data
# 110
dfa = pd.read_csv('excel/110.csv',encoding = 'utf-8')

month = []
for i in dfa['日期']:
     month.append(int(i.split('月')[0]))
data = {'month': month,
        'water_in': dfa['水庫進流量/立方公尺'].str.replace(",","").astype(int)}
df1 = pd.DataFrame(data)

df2 = pd.DataFrame(df1.groupby('month').mean())
water_in = np.array(df2["water_in"])
month = np.array(df2.index)

data2 = {'year': 108,
        'month': month,
        'water_in': water_in}
df110 = pd.DataFrame(data2)

# 109
dfa = pd.read_csv('excel/109.csv',encoding = 'utf-8')

month = []
for i in dfa['日期']:
     month.append(int(i.split('月')[0]))
data = {'month': month,
        'water_in': dfa['水庫進流量/立方公尺'].str.replace(",","").astype(int)}
df1 = pd.DataFrame(data)

df2 = pd.DataFrame(df1.groupby('month').mean())
water_in = np.array(df2["water_in"])
month = np.array(df2.index)

data2 = {'year': 109,
        'month': month,
        'water_in': water_in}
df109 = pd.DataFrame(data2)

#108
dfa = pd.read_csv('excel/108.csv',encoding = 'utf-8')

month = []
for i in dfa['日期']:
     month.append(int(i.split('月')[0]))
data = {'month': month,
        'water_in': dfa['水庫進流量/立方公尺'].str.replace(",","").astype(int)}
df1 = pd.DataFrame(data)

df2 = pd.DataFrame(df1.groupby('month').mean())
water_in = np.array(df2["water_in"])
month = np.array(df2.index)

data2 = {'year': 110,
        'month': month,
        'water_in': water_in}
df108 = pd.DataFrame(data2)

# final data
frames = [df108, df109, df110]
dff = pd.concat(frames)
dff.reset_index(drop=True, inplace=True)
display(dff)

In [ ]:
# Regression
Month_dummy = pd.get_dummies(dff['month'], prefix = 'month')
df20_3d = pd.concat([dff, Month_dummy], axis = 1)
df20_3d.head()

y_data_df20_3d = dff['water_in']
td_r = np.arange(0, len(dff['water_in']), 1)
X_data_ar_df20_3d = np.array([td_r, df20_3d['month_1'], df20_3d['month_2'], df20_3d['month_3'], df20_3d['month_4'], df20_3d['month_5'], df20_3d['month_6']
                              , df20_3d['month_7'], df20_3d['month_8'], df20_3d['month_9'], df20_3d['month_10'], df20_3d['month_11']])
X_data_ar_df20_3d_T = X_data_ar_df20_3d.T
X_data_df20_3d = pd.DataFrame(X_data_ar_df20_3d_T, columns = ['t', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6'
                                                              , 'month_7', 'month_8', 'month_9', 'month_10', 'month_11'])
X_data_df20_3d = sm.add_constant(X_data_df20_3d)
olsmod_df20_3d_DM = sm.OLS(y_data_df20_3d, X_data_df20_3d)
result_reg_df20_3d_DM = olsmod_df20_3d_DM.fit()
print(result_reg_df20_3d_DM.summary())

df20_3d_DM_st, df20_3d_DM_data, df20_3d_DM_ss2 = sso.summary_table(result_reg_df20_3d_DM, alpha=0.05)
df20_3d_DM_y_pre = df20_3d_DM_data[:, 2]
df20_3d_DM_SD = df20_3d_DM_data[:, 10]
des_pre_df20_3d_DM = df20_3d_DM_y_pre
print('Fitted Values', des_pre_df20_3d_DM)

# Line Chart
plt.plot(td_r, dff['water_in'])
plt.plot(td_r, des_pre_df20_3d_DM)
plt.legend(labels=['orig', 'Line_pre'])
plt.xlabel('time')
plt.ylabel('water_in')
plt.title('Water In by LR Dummy Variables')
plt.show()

# Coefficient of Correlation
df1_2 = X_data_df20_3d[['t', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11']]
corr1 = df1_2.corr()
corr1
_ = sns.heatmap(corr1, annot=True)

<h2>Perform the residual analysis:

In [ ]:
# Standardized Residuals
st, data, ss2 = sso.summary_table(result_reg_df20_3d_DM, alpha=0.05)
print("Columns in data are: %s" % ss2)
#Predicted value
y_pre = data[:, 2]
#Studentized Residual
SD = data[:, 10]

Test 1: Normality Test

Hypothesis:<br>
H0: Errors are normally distributed.<br>
H1: Errors are not normally distributed.<br>

Because p_value = 0.180 > a = 0.05, do not rejected H0. We can assume that errors are normally distributed.

Test 2: Homoscedasticity and Heteroscedasticity:

Hypothesis:<br>
H0: Homoscedasticity<br>
H1: Heteroscedasticity

In [ ]:
#Test 2: Homoscedasticity and Heteroscedasticity:
plt.plot(y_pre, SD, 'o', color = 'gray')
plt.axhline(y=2, color = 'red', lw = 0.8)
plt.axhline(y=0, color = 'blue')
plt.axhline(y=-2, color = 'red', lw = 0.8)
plt.title('Standardized Residual Plot')
plt.xlabel('Predicted y value')
plt.ylabel('Standardized Residual')
plt.show()

Do not rejected H0. We can assume that the variation is constant and the mean is around 0.

Test 3: Dependence of the Error Variable:<br>

Hypothesis:<br>
H0 : Randomness exists.<br>
H1 : Randomness does not exist.

Because p-value = 0.3102968672638191 > a = 0.05, do not reject H0.
There is no evidence to infer that the sample is not random.

All required conditions satisfied.

Multicollinearity analysis:

According to the OLS Regression Results chart and the correlation table above.<br>
The model is valid (F-statistic pvalue = 0.00132 < 0.05 rejects null hypothesis, r-squared is 0.242).<br>
Lot size (0.216), Distance (0.058) p-value of T-test rejects null hypothesis, Trees (0.004) p-value of T-test do not rejects null hypothesis.<br>
The sign of the correlations are the same as the coefficents.<br>
Thus, there is no multicollinearity issue in this model.

Right-tail test for First Order Auto-Correlation:<br>
H0: The data are not first-order correlated<br>
H1: The data are first-order correlated

(T = 36, K = 12, alpha = 0.05): dl = 0.748 , du = 2.398<br>??
2 - dl < d = 1.5667996478416744 < 2. Thus, there is not enough evidence to infer that first order auto-correlation exists.

<h2>#############################

<h2>Predict

New_t = np.array([[20, 1, 0, 0],[21, 0, 1, 0], [22, 0, 0, 1], [23, 0, 0, 0]])
Total_t = np.append(td_r, New_t[:,0])
New_t = sm.add_constant(New_t)
df20_3d_DM_pre_New_t = result_reg_df20_3d_DM.predict(New_t)
print('New Dummy Season Prediction', df20_3d_DM_pre_New_t)
New_Nan = np.zeros(New_t.shape[0])
New_Nan[:] = np.nan
df20_3d_DM_New_orig = np.append(df20_3['Rate'], New_Nan)
df20_3d_DM_New_pre = np.append(des_pre_df20_3d_DM, df20_3d_DM_pre_New_t)
plt.plot(Total_t, df20_3d_DM_New_orig)
plt.plot(Total_t, df20_3d_DM_New_pre)
plt.legend(labels=['org', 'Line_pre'])
plt.xlabel('time')
plt.ylabel('Sales')
plt.title('Hotel Occupancy Rates by LR Dummy Variables')
plt.show()

# SLR

In [ ]:
# Reading dataset
# 讀取資料集
df108 = pd.read_csv('excel/108.csv',encoding = 'utf-8')

# 簡單看一下資料集的樣子

print('Head of dataset:')
display(df108.head())

print("Tail of dataset:")
display(df108.tail())

In [ ]:
for i in range(365):
    df108['水庫進流量/立方公尺'][i] = int(df108['水庫進流量/立方公尺'][i].replace(",",""))
df108['水庫進流量/立方公尺']

In [ ]:
df108data = []
df108data.append(df108['水庫進流量/立方公尺'].iloc[:31].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[31:59].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[59:90].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[90:120].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[120:151].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[151:181].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[181:212].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[212:243].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[243:273].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[273:304].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[304:334].mean())
df108data.append(df108['水庫進流量/立方公尺'].iloc[334:365].mean())
for i in range(12):
    df108data[i] = round(df108data[i])
    
len(df108data)

In [ ]:
# Reading dataset
# 讀取資料集
df109 = pd.read_csv('excel/109.csv',encoding = 'utf-8')

# 簡單看一下資料集的樣子

print('Head of dataset:')
display(df109.head())

print("Tail of dataset:")
display(df109.tail())

In [ ]:
for i in range(df109.shape[0]):
    df109['水庫進流量/立方公尺'][i] = int(df109['水庫進流量/立方公尺'][i].replace(",",""))
df109['水庫進流量/立方公尺']

In [ ]:
df109data = []
df109data.append(df109['水庫進流量/立方公尺'].iloc[:31].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[31:60].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[60:91].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[91:121].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[121:152].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[152:182].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[182:213].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[213:244].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[244:274].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[274:305].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[305:335].mean())
df109data.append(df109['水庫進流量/立方公尺'].iloc[335:366].mean())
for i in range(12):
    df109data[i] = round(df109data[i])
    
df109data

In [ ]:
# Reading dataset
# 讀取資料集
df110 = pd.read_csv('excel/110.csv',encoding = 'utf-8')

# 簡單看一下資料集的樣子

print('Head of dataset:')
display(df110.head())

print("Tail of dataset:")
display(df110.tail())

In [ ]:
for i in range(df110.shape[0]):
    df110['水庫進流量/立方公尺'][i] = int(df110['水庫進流量/立方公尺'][i].replace(",",""))
df110['水庫進流量/立方公尺']

In [ ]:
df110data = []
df110data.append(df110['水庫進流量/立方公尺'].iloc[:31].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[31:59].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[59:90].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[90:120].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[120:151].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[151:181].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[181:212].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[212:243].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[243:273].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[273:304].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[304:334].mean())
df110data.append(df110['水庫進流量/立方公尺'].iloc[334:365].mean())
for i in range(12):
    df110data[i] = round(df110data[i])
    
df110data

In [ ]:
totalData = df108data + df109data + df110data
print(len(totalData))
print(totalData)

In [ ]:
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False
# 上面這兩句有設應該就行了
totalData_SILR = pd.DataFrame(columns=["LR_ind", "SI_LR_res"])
totalData_SILR = SI_LR(totalData, 12)
totalData_LR_Idx = totalData_SILR["LR_ind"]
print(totalData_LR_Idx)
totalData_LR_Result = totalData_SILR["SI_LR_res"]
display(totalData_LR_Result)
t_r = np.arange(0, len(totalData), 1)
plt.plot(t_r, totalData_LR_Result['orig'])
plt.plot(t_r, totalData_LR_Result['Des_D'])
plt.legend(labels=['org', 'Des_data'])
plt.xlabel('time')
plt.ylabel('月平均水庫進流量/立方公尺')
plt.title('月平均水庫進流量/立方公尺')
plt.show()

In [ ]:
y_data_totalData_LR = totalData_LR_Result['Des_D']
X_data_ar_totalData = np.array(t_r)
X_data_ar_totalData_T = X_data_ar_totalData.T
X_data_totalData = pd.DataFrame(X_data_ar_totalData_T, columns = ['t'])
X_data_totalData = sm.add_constant(X_data_totalData)
olsmod_totalData_LR = sm.OLS(y_data_totalData_LR, X_data_totalData)
result_reg_totalData_LR = olsmod_totalData_LR.fit()
print(result_reg_totalData_LR.summary())
totalData_LR_st_1, totalData_LR_data_1, totalData_LR_ss2_1 = sso.summary_table(result_reg_totalData_LR, alpha=0.05)
totalData_LR_y_pre_1 = totalData_LR_data_1[:, 2]
totalData_LR_SD_1 = totalData_LR_data_1[:, 10]
des_pre_totalData_LR = totalData_LR_y_pre_1
print(des_pre_totalData_LR)
plt.plot(t_r, totalData_LR_Result['orig'])
plt.plot(t_r, totalData_LR_Result['Des_D'])
plt.plot(t_r, des_pre_totalData_LR)
plt.legend(labels=['org', 'Des_data', 'Line_pre'])
plt.xlabel('time')
plt.ylabel('月平均水庫進流量/立方公尺')
plt.title('月平均水庫進流量/立方公尺 by Linear Regression Seasonal Index')
plt.show()

## Residual Analysis

In [ ]:
totalData_st, totalData_data, totalData_ss2 = sso.summary_table(result_reg_totalData_LR, alpha=0.05)
totalData_y_pre = totalData_data[:, 2]
totalData_SD = totalData_data[:, 10]
print(totalData_SD)
plt.plot(totalData_y_pre, totalData_SD, 'o', color = 'gray')
plt.axhline(y=2, color = 'red', lw = 0.8)
plt.axhline(y=0, color = 'blue')
plt.axhline(y=-2, color = 'red', lw = 0.8)
plt.title('Standardized Residual Plot')
plt.xlabel('Predicted y value')
plt.ylabel('Standardized Residual')
plt.show()
print('Shapiro Test')
stat, p = stats.shapiro(totalData_SD)
print('Statistics=%.3f, p=%.3f' % (stat, p))
totalData_SD_median = statistics.median(totalData_SD)
#print('median = ', SD_median)
Z_pval = runsTest(totalData_SD, totalData_SD_median) 
print('p_value for Z-statistic= ', Z_pval)
print('\n')
print('The Durbin Watson test')
d_w = Durbin_Watson_test(totalData_SD)
print('d value =', d_w)

n = 36, k = 1, alpha = 0.05

dL=1.411, dU=1.525
Because d value 1.1138025223787504 < dL = 1.411, the first order correlation exists.

## Forecasting

In [ ]:
Sea_pre_totalData_LR = des_pre_totalData_LR * totalData_LR_Result['SeaIdx']
New_t_1 = np.array([36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])
Total_t_1 = np.append(t_r, New_t_1)
New_t_1 = sm.add_constant(New_t_1)
totalData_LR_des_pre_New_t_1 = result_reg_totalData_LR.predict(New_t_1)
print('New Des Prediction', totalData_LR_des_pre_New_t_1)
totalData_LR_Sea_pre_New_t_1 = totalData_LR_des_pre_New_t_1 * totalData_LR_Idx
print('New Seasonal Prediction', totalData_LR_Sea_pre_New_t_1)
New_Nan = np.zeros(New_t_1.shape[0])
New_Nan[:] = np.nan
totalData_LR_New_orig_1 = np.append(totalData_LR_Result['orig'], New_Nan)
totalData_LR_New_Des_D_1 = np.append(totalData_LR_Result['Des_D'], New_Nan)
totalData_LR_New_des_pre_1 = np.append(des_pre_totalData_LR, totalData_LR_des_pre_New_t_1)
totalData_LR_New_Sea_pre_1 = np.append(Sea_pre_totalData_LR, totalData_LR_Sea_pre_New_t_1)
totalData_LR_S_a = np.array([Total_t_1, totalData_LR_New_orig_1, totalData_LR_New_Des_D_1, totalData_LR_New_des_pre_1, totalData_LR_New_Sea_pre_1])
totalData_LR_S_df = pd.DataFrame(totalData_LR_S_a.T, columns=['time', 'org', 'Des_data', 'Line_pre', 'Seasonal Pre'])
display(totalData_LR_S_df)
plt.plot(Total_t_1, totalData_LR_New_orig_1)
plt.plot(Total_t_1, totalData_LR_New_Des_D_1)
plt.plot(Total_t_1, totalData_LR_New_des_pre_1)
plt.plot(Total_t_1, totalData_LR_New_Sea_pre_1)
plt.legend(labels=['org', 'Des_data', 'Line_pre', 'Seasonal Pre'])
plt.xlabel('time')
plt.ylabel('月平均水庫進流量/立方公尺')
plt.title('月平均水庫進流量/立方公尺 by Linear Regression Seasonal Index')
plt.show()

In [ ]:
print('Error Metrics for Seasonal Index by Moving Average')
ch20_ts3_MA_Err_M = ErrMetrix(df_1['mean'], Sea_pre_df_1_ts3_MA)
print('\n')
print('Error Metrics for Seasonal Index by LR')
ch20_ts3_LR_Err_M = ErrMetrix(totalData, Sea_pre_totalData_LR)
print('\n')
print('Error Metrics for LR Dummy')
ch20_ts3d_DM_Err_M = ErrMetrix(dff['water_in'], des_pre_df20_3d_DM)
ErrM_id = ['MAD', 'MSE', 'RMSE', 'MAPE']
ch20_ts3_ErrM_a = np.array([ErrM_id, ch20_ts3_MA_Err_M, ch20_ts3_LR_Err_M, ch20_ts3d_DM_Err_M])
ch20_ts3_ErrM_df = pd.DataFrame(ch20_ts3_ErrM_a.T, columns = ['ErrM', 'SIMA', 'SILR', 'Dummy'])
print('dataset:')
display(ch20_ts3_ErrM_df)